In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage import exposure

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_shape = (224, 224, 3)
num_classes = 10

In [ ]:
train_ds_path = '/content/drive/MyDrive/KKDL/dataset_classifier/training'
val_ds_path = '/content/drive/MyDrive/KKDL/dataset_classifier/validation'
test_ds_path = '/content/drive/MyDrive/KKDL/dataset_classifier/testing'

In [ ]:
train_ds = image_dataset_from_directory(
    train_ds_path,
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(input_shape[0], input_shape[1]),
    shuffle=True,
    interpolation="bilinear",
)

val_ds = image_dataset_from_directory(
    val_ds_path,
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(input_shape[0], input_shape[1]),
    shuffle=True,
    interpolation="bilinear",
)

test_ds = image_dataset_from_directory(
    test_ds_path,
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(input_shape[0], input_shape[1]),
    shuffle=False,
    interpolation="bilinear",
)

Found 1944 files belonging to 10 classes.
Found 699 files belonging to 10 classes.
Found 100 files belonging to 10 classes.


In [ ]:
def extract_hog_features(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (32, 32))  # Thay đổi kích thước hình ảnh thành kích thước cố định
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Áp dụng trích đặc trưng HOG
    fd, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    # Thay đổi kích thước các tính năng HOG để hiển thị tốt hơn
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd

In [ ]:
# Hình ảnh trong thư mục được chỉ định và trích xuất các đặc trưng HOG từ mỗi hình ảnh
def prepare_data(directory):
    features = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            features.append(extract_hog_features(image_path))
            labels.append(class_name)
    return np.array(features), np.array(labels)


In [ ]:
train_features, train_labels = prepare_data(train_ds_path)
val_features, val_labels = prepare_data(val_ds_path)

In [ ]:
# Huấn luyện mô hình KNN
model = KNeighborsClassifier(n_neighbors=1)

model.fit(train_features, train_labels)

val_predictions = model.predict(val_features)

# In ra độ chính xác của mô hình
accuracy = accuracy_score(val_labels, val_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5994277539341917


In [ ]:
# Example usage:
new_image_path = '/content/drive/MyDrive/KKDL/dataset_classifier/testing/fruit-bores/fruit-bores_32.jpg'
predicted_label = predict_image(new_image_path, model)
print("Predicted label:", predicted_label)

Predicted label: fruit-bores


In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt

def predict_and_verify(model, class_names, image_path):
    global WIDTH
    global HEIGHT

    img = image.load_img(image_path, target_size=(WIDTH, HEIGHT))  # Thay đổi kích thước theo kiến trúc LeNet

    # Chuyển ảnh thành mảng NumPy
    img_array = image.img_to_array(img)

    # Chuẩn hóa giá trị pixel về khoảng [0, 1]
    img_array /= 255.0

    # Thêm một chiều để tạo thành batch (nếu cần)
    img_array = np.expand_dims(img_array, axis=0)

    # Dự đoán lớp của ảnh
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class_name = list(class_names.keys())[predicted_class_index]

    # In ra kết quả dự đoán và xác suất của các lớp
    print(f"Predicted class: {predicted_class_name}")
    print(f"Class probabilities: {predictions[0]}")

    # Hiển thị ảnh để kiểm tra
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Sử dụng hàm để dự đoán và xác minh kết quả
image_path_to_verify = '/content/drive/MyDrive/KKDL/dataset_classifier/testing/fruit-bores/fruit-bores_32.jpg'
# predict_and_verify(model, train_generator.class_indices, image_path_to_verify)
class_names = {
    'fruit-bores': 0,
    'grasshopper': 1,
    'leafroller': 2,
    'maybug': 3,
    'mealybug-disease': 4,
    'moth': 5,
    'red-spider': 6,
    'snail': 7,
    'stag-beetle': 8,
    'stinkbug': 9
}
predict_and_verify(model, class_names, image_path_to_verify)